In [1]:
import chromedriver_autoinstaller
from selenium import webdriver

In [2]:
opt = webdriver.ChromeOptions()
opt.add_argument("--start-maximized")

chromedriver_autoinstaller.install()

'C:\\Users\\H P\\Envs\\scrapperenv\\lib\\site-packages\\chromedriver_autoinstaller\\101\\chromedriver.exe'

In [3]:
driver = webdriver.Chrome(options=opt)

In [4]:
url = "https://apps.colorado.gov/dora/licensing/Lookup/LicenseLookup.aspx"
driver.get(url)

In [23]:
failed = {}

In [25]:
import os
import time
import string
import pandas as pd

TABLE_ID = "ctl00_MainContentPlaceHolder_ucLicenseLookup_gvSearchResults"
FOLDER   = "records"
P1_COLS  = ["Name", "License Number", "License Status", "Contact Type", "City", "State", "Zip Code"]
P2G0_COLS= ["P2Name", "Public Address"]
P2G1_COLS= ["P2License Number", "License Method", "License Type", "P2License Status", "Original Issue Date", "Effective Date", "Expiration Date"]

def extract_popup2(handler, r):
    while True:
        try:
            tds = handler.find_element_by_id("Grid0").find_element_by_tag_name("tbody").find_elements_by_tag_name("td")
            break
        except Exception as e:
            time.sleep(1)
    for i, td in enumerate(tds):
        if i < len(P2G0_COLS):
            r[P2G0_COLS[i]] = td.text
        
    tds = handler.find_element_by_id("Grid1").find_element_by_tag_name("tbody").find_elements_by_tag_name("td")
    for i, td in enumerate(tds):
        if i < len(P2G1_COLS):
            r[P2G1_COLS[i]] = td.text
        
    
def extract_record(handler):
    records= []
    for i in range(3):
        try:
            trows = handler.find_element_by_id(TABLE_ID).find_element_by_tag_name("tbody").find_elements_by_tag_name("tr")
            if len(trows) > 0:
                break
        except Exception as e:
            time.sleep(1)
            
    for trow in trows:
        r = {}
        tds = trow.find_elements_by_tag_name("td")
        detail_btn = tds[0].find_element_by_tag_name("a")
        tds = tds[1:]
        for i, c in enumerate(P1_COLS):
            if i < len(tds):
                r[c] = tds[i].text
            else:
                r[c] = ""
        
        # extract popup 2
        detail_btn.click()
        time.sleep(2)
        extract_popup2(handler, r)
        records.append(r)
        
        while True:
            try:
                handler.find_element_by_class_name("k-window-actions").find_elements_by_tag_name("a")[-1].click()
                break
            except Exception as e:
                time.sleep(1)
        
    return records
        

def save_records(char, records):
    if not os.path.exists(FOLDER):
        os.makedirs(FOLDER)
    df = pd.DataFrame(records)
    df.to_excel(os.path.join(FOLDER, f"{char}.xlsx"),  index=False)

to_ignore = []
for ch in ['y', 'z']:#string.ascii_lowercase:
    if ch in to_ignore:
        continue
    data_items = []
    
    # set input &  lookup for records
    fname = driver.find_element_by_id("ctl00_MainContentPlaceHolder_ucLicenseLookup_ctl03_tbFirstName_Contact")
    driver.execute_script(f"arguments[0].value = '{ch}';", fname)
    driver.find_element_by_id("btnLookup").click()
    time.sleep(2)
    
    # fetch records from first popup
    while True:
        try:
            pages      = driver.find_element_by_id(TABLE_ID).find_element_by_tag_name("thead").find_elements_by_tag_name("a")
            break
        except Exception as e:
            time.sleep(1)
    page_count = len(pages) 
    items      = extract_record(driver)
    data_items.extend(items)
    
    for i in range(page_count):
        try:
            page      = driver.find_element_by_id(TABLE_ID).find_element_by_tag_name("thead").find_elements_by_tag_name("a")[i]
            page.click()
            time.sleep(2)
            items = extract_record(driver)
            data_items.extend(items)
        except:
            if ch in failed.keys():
                failed[ch].append(i+1)
            else:
                failed[ch] = [i+1]
        
    driver.find_element_by_class_name("close").click()
    save_records(ch, data_items)
    
    

C:\Users\H P\Envs\scrapperenv\lib\site-packages\ipykernel_launcher.py:79: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
C:\Users\H P\Envs\scrapperenv\lib\site-packages\ipykernel_launcher.py:81: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
C:\Users\H P\Envs\scrapperenv\lib\site-packages\ipykernel_launcher.py:87: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
C:\Users\H P\Envs\scrapperenv\lib\site-packages\ipykernel_launcher.py:33: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
C:\Users\H P\Envs\scrapperenv\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  from ipykernel import kernelapp as app
C:\Users\H P\Envs\scrapperenv\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: find_eleme

In [41]:
save_records(ch, data_items)

In [16]:
len(data_items)

12

In [26]:
failed

{'c': [2],
 'g': [4],
 'j': [5],
 'k': [8],
 'l': [7],
 'p': [6],
 's': [3, 5],
 'v': [1, 3],
 'w': [5]}

In [5]:
fname = driver.find_element_by_id("ctl00_MainContentPlaceHolder_ucLicenseLookup_ctl03_tbFirstName_Contact")
fname

<selenium.webdriver.remote.webelement.WebElement (session="8b9c70ca22e57b924a3b11a2e0ea984c", element="063fe254-0e96-4e9e-a31b-5350671dca47")>

In [6]:
driver.execute_script("arguments[0].value = 'a';", fname)

In [7]:
search_btn = driver.find_element_by_id("btnLookup")
search_btn

<selenium.webdriver.remote.webelement.WebElement (session="8b9c70ca22e57b924a3b11a2e0ea984c", element="4a225b58-cd44-4cd9-87a8-6212405656ee")>

In [8]:
search_btn.click()

In [9]:
table = driver.find_element_by_id("ctl00_MainContentPlaceHolder_ucLicenseLookup_gvSearchResults")
table

<selenium.webdriver.remote.webelement.WebElement (session="8b9c70ca22e57b924a3b11a2e0ea984c", element="4d8e19f4-0459-49ad-a463-705742382316")>

In [10]:
thead = table.find_element_by_tag_name("thead")
thead

<selenium.webdriver.remote.webelement.WebElement (session="8b9c70ca22e57b924a3b11a2e0ea984c", element="371825ab-1fe0-451d-936f-f14d0a4261c8")>

In [11]:
tbody = table.find_element_by_tag_name("tbody")
tbody

<selenium.webdriver.remote.webelement.WebElement (session="8b9c70ca22e57b924a3b11a2e0ea984c", element="32ca5d60-9ccb-4826-abf1-fc4059868513")>

In [12]:
atags = thead.find_elements_by_tag_name("a")
len(atags)

8

In [13]:
trows = tbody.find_elements_by_tag_name("tr")
len(trows)

12

In [14]:
cols = ["Name", "License Number", "License Status", "Contact Type", "City", "State", "Zip Code"]
data_items = []
detail_btns= []
for trow in trows:
    d = {}
    tds = trow.find_elements_by_tag_name("td")
    detail_btn = tds[0].find_element_by_tag_name("a")
    tds = tds[1:]
    for i, c in enumerate(cols):
        d[c] = tds[i].text
    detail_btns.append(detail_btn)
    data_items.append(d)

In [15]:
len(data_items)

12

In [16]:
dbtn = detail_btns[0]
dbtn

<selenium.webdriver.remote.webelement.WebElement (session="8b9c70ca22e57b924a3b11a2e0ea984c", element="cd1870f3-a3fb-4187-8932-8ae60d64ee41")>

In [30]:
dbtn.text

'Detail'

In [17]:
dbtn.click()

In [18]:
cols = ["P2Name", "Public Address"]
grid0 = driver.find_element_by_id("Grid0")
ds = []
for i, td in enumerate(grid0.find_element_by_tag_name("tbody").find_elements_by_tag_name("td")):
    d = {}
    d[cols[i]] = td.text
    ds.append(d)
ds

[{'P2Name': 'Angela Dawn Abbott'},
 {'Public Address': 'Colorado Springs, CO 80916-2439'}]

In [19]:
grid1 = driver.find_element_by_id("Grid1")
cols = ["P2License Number", "License Method", "License Type", "P2License Status", "Original Issue Date", "Effective Date", "Expiration Date"]
ds = []
for i, td in enumerate(grid1.find_element_by_tag_name("tbody").find_elements_by_tag_name("td")):
    d = {}
    d[cols[i]] = td.text
    ds.append(d)
ds

[{'P2License Number': 'VET.0010816'},
 {'License Method': 'Examination'},
 {'License Type': 'Veterinarian'},
 {'P2License Status': 'Active'},
 {'Original Issue Date': '11/18/2016'},
 {'Effective Date': '11/01/2020'},
 {'Expiration Date': '10/31/2022'}]

In [20]:
btnclose = driver.find_element_by_class_name("k-window-actions").find_elements_by_tag_name("a")[-1]
btnclose

<selenium.webdriver.remote.webelement.WebElement (session="8b9c70ca22e57b924a3b11a2e0ea984c", element="27c719d2-c629-4642-a956-17b741eb0548")>

In [21]:
btnclose.click()

In [22]:
btnclose1 = driver.find_element_by_class_name("close")
btnclose1

<selenium.webdriver.remote.webelement.WebElement (session="8b9c70ca22e57b924a3b11a2e0ea984c", element="cd981781-f14e-41f6-b067-a6f063336ae6")>

In [23]:
btnclose1.click()